File Format Converter 
- To Use Tiff images for inference, it should follow predefined format (Refer to nnUnet Official Documents)

In [3]:
import os
import glob
import json
import shutil
import subprocess
import cv2
import numpy as np
import nibabel as nib
from tqdm import tqdm

# ================================================================
# PATH CONFIG
# ================================================================
ORIG_TIF_DIR = "/shared/data/climateplus2025/CapeTown_Image_2023_3samples_poster_1024size_Nov20/original_cropped_images"

NNUNET_FORMAT_DIR = "/shared/data/climateplus2025/CapeTown_Image_2023_3samples_poster_1024size_Nov20/original_cropped_images_nnUNet_Format"

PRED_ROOT = "/shared/data/climateplus2025/Prediction_for_poster_3_images_nnUnet_1024_Nov21"
PRED_NIFTI = os.path.join(PRED_ROOT, "pred_raw_nnunet")
PRED_PNG   = os.path.join(PRED_ROOT, "pred_png")
PRED_JSON  = os.path.join(PRED_ROOT, "pred_json")

for d in [NNUNET_FORMAT_DIR, PRED_NIFTI, PRED_PNG, PRED_JSON]:
    os.makedirs(d, exist_ok=True)

# ================================================================
# SET REQUIRED ENVIRONMENT VARIABLES
# ================================================================
os.environ["nnUNet_raw"] = "/shared/data/climateplus2025/NNUNET_TEMP_raw"
os.environ["nnUNet_preprocessed"] = "/shared/data/climateplus2025/NNUNET_TEMP_preprocessed"
os.environ["nnUNet_results"] = "/shared/data/climateplus2025/Shawn_Nov18_NNUnet_1024/nnUNet_results"

os.makedirs(os.environ["nnUNet_raw"], exist_ok=True)
os.makedirs(os.environ["nnUNet_preprocessed"], exist_ok=True)


# ================================================================
# COLOR MAP (consistent with Mask2Former)
# ================================================================
COLORS = {
    0: (0,0,0),
    1: (0,255,0),
    2: (0,0,255),
    3: (255,0,0),
}

def mask_to_color(mask):
    h, w = mask.shape
    out = np.zeros((h, w, 3), dtype=np.uint8)
    for k, rgb in COLORS.items():
        out[mask == k] = rgb
    return out

def coords_from_mask(base, mask):
    result = {"image_name": f"{base}.png", "predicted_coords": {}}
    for cls in [1,2,3]:
        ys, xs = np.where(mask == cls)
        if len(ys) > 0:
            result["predicted_coords"][f"class_{cls}"] = np.stack([ys, xs], axis=1).tolist()
    return result


# ================================================================
# STEP 1 — Copy TIFFs → nnU-Net naming format
# ================================================================
def prepare_nnunet_tiff_inputs():
    print("\n=== STEP 1: Preparing TIFF files for nnU-Net ===")

    tif_files = sorted(glob.glob(os.path.join(ORIG_TIF_DIR, "*.tif")))
    print(f"Found {len(tif_files)} TIFF files")

    for idx, src in tqdm(enumerate(tif_files), total=len(tif_files)):
        # Create nnUNet valid name
        # Example: image_000123_0000.tif
        base = os.path.splitext(os.path.basename(src))[0]
        dst_name = f"{base}_0000.tif"
        dst = os.path.join(NNUNET_FORMAT_DIR, dst_name)

        shutil.copy(src, dst)

    print("✓ TIFF copied & renamed for nnU-Net.")


# ================================================================
# STEP 2 — Run nnU-Net inference
# ================================================================
def run_nnunet():
    print("\n=== STEP 2: nnU-Net Inference ===")

    cmd = [
        "nnUNetv2_predict",
        "-i", NNUNET_FORMAT_DIR,
        "-o", PRED_NIFTI,
        "-d", "901",
        "-c", "2d",
        "-f", "0", "1", "2", "3", "4"
    ]

    print("Running:", " ".join(cmd))
    subprocess.run(cmd, check=True)

    print("✓ nnU-Net prediction completed.")


# ================================================================
# STEP 3 — Convert NIfTI → PNG + JSON
# ================================================================
def export_outputs():
    print("\n=== STEP 3: Export PNG & JSON ===")

    nii_files = sorted(glob.glob(os.path.join(PRED_NIFTI, "*.nii.gz")))
    print(f"Converting {len(nii_files)} predicted NIfTI files...\n")

    for path in tqdm(nii_files):
        base = os.path.basename(path).replace(".nii.gz", "")
        nii = nib.load(path)
        arr = nii.get_fdata().astype(np.uint8)

        if arr.ndim == 3:
            arr = arr.squeeze()

        # --- PNG ---
        color = mask_to_color(arr)
        cv2.imwrite(
            os.path.join(PRED_PNG, f"{base}.png"),
            cv2.cvtColor(color, cv2.COLOR_RGB2BGR)
        )

        # --- JSON ---
        coords = coords_from_mask(base, arr)
        with open(os.path.join(PRED_JSON, f"{base}.json"), "w") as fp:
            json.dump(coords, fp, separators=(",", ":"), ensure_ascii=False)

    print("✓ PNG + JSON export completed.")


# ================================================================
# MAIN
# ================================================================
if __name__ == "__main__":
    prepare_nnunet_tiff_inputs()
    run_nnunet()
    export_outputs()
    print("\n=== ALL DONE — JSON & PNG READY ===")


=== STEP 1: Preparing TIFF files for nnU-Net ===
Found 507 TIFF files


100%|██████████| 507/507 [00:15<00:00, 32.40it/s]


✓ TIFF copied & renamed for nnU-Net.

=== STEP 2: nnU-Net Inference ===
Running: nnUNetv2_predict -i /shared/data/climateplus2025/CapeTown_Image_2023_3samples_poster_1024size_Nov20/original_cropped_images_nnUNet_Format -o /shared/data/climateplus2025/Prediction_for_poster_3_images_nnUnet_1024_Nov21/pred_raw_nnunet -d 901 -c 2d -f 0 1 2 3 4

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



/home/il72/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


There are 0 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 0 cases that I would like to predict
✓ nnU-Net prediction completed.

=== STEP 3: Export PNG & JSON ===
Converting 0 predicted NIfTI files...



0it [00:00, ?it/s]

✓ PNG + JSON export completed.

=== ALL DONE — JSON & PNG READY ===
